In [13]:
#https://drive.google.com/drive/folders/1GAwZryVvNeG6U9Gezri839y9Q1fWJLBM?usp=sharing


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import duckdb

file_path = '/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/Data.csv'
df = pd.read_csv(file_path, delimiter='\t')

print(df.head())

   Sub            Timestamp  Value Event_name  Rank  Post_5  Pre_5  Post_10  \
0    1  10/31/2024 22:21:30      0        NaN     0       0      0        0   
1    1  10/31/2024 22:21:31      0        NaN     0       0      0        0   
2    1  10/31/2024 22:21:32      0        NaN     0       0      0        0   
3    1  10/31/2024 22:21:33      0        NaN     0       0      0        0   
4    1  10/31/2024 22:21:34      0        NaN     0       0      0        0   

   Pre_10  Sys_median Sys_mode  O_median Pos_median  HR_median  Di_median  
0       0       189.0     Wake       0.0     Supine       83.5      125.0  
1       0       189.5     Wake       0.0     Supine       83.0      124.0  
2       0       188.0     Wake      92.0     Supine       83.0      121.0  
3       0       183.0     Wake      92.0     Supine       83.0      120.0  
4       0       179.0     Wake      92.0     Supine       83.0      117.5  


In [3]:


duckdb.query("""
    SELECT  distinct Event_name
    FROM df
    
""")

┌──────────────────────┐
│      Event_name      │
│       varchar        │
├──────────────────────┤
│ Body event           │
│ Hypopnea             │
│ SpO2 Systolic        │
│ NULL                 │
│ Obstructive Apnea    │
│ PLM Systolic         │
│ Mixed Apnea          │
│ Cardiac Systolic     │
│ Central Apnea        │
│ Spontaneous Systolic │
├──────────────────────┤
│       10 rows        │
└──────────────────────┘

In [4]:


duckdb.query("""
    SELECT  Sub, Event_name, count(distinct Rank) as events
    FROM df
    where
        Event_name <> 'None'
    group by
        Sub, Event_name
    
""").df().to_csv('/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/tmp/events_summary.tsv', sep='\t', index=False)

In [3]:
#1. Event data

In [13]:
duckdb.query("""
    WITH enriched AS (
        SELECT
            *,
            FIRST_VALUE(Sys_mode) OVER (
                PARTITION BY Sub, Event_name, Rank 
                ORDER BY Timestamp DESC
            ) AS event_Sys_mode,

            FIRST_VALUE(Pos_median) OVER (
                PARTITION BY Sub, Event_name, Rank 
                ORDER BY Timestamp DESC
            ) AS event_Pos_median
        FROM df
        WHERE Event_name IN ('Obstructive Apnea', 'Hypopnea')
    )

    SELECT
        Sub,
        Event_name,
        Rank,
        MIN(Timestamp) AS Timestamp,
        COUNT(DISTINCT Timestamp) AS event_duration,
        event_Sys_mode,
        event_Pos_median,
        AVG(Sys_median)     AS event_mean_Systolic,
        AVG(O_median)       AS event_mean_SpO2,
        AVG(HR_median)      AS event_mean_HeartRate,
        AVG(Di_median)      AS event_mean_Diastolic
    FROM enriched
    GROUP BY
        Sub,
        Event_name,
        Rank,
        event_Sys_mode,
        event_Pos_median
""").df().to_csv(
    '/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/tmp/event.tsv',
    sep='\t',
    index=False
)




In [40]:
#2. PrezEvent data

duckdb.query("""
-- 1)  DISTINCT (Sub, Rank) for the two target event types
WITH target_ranks AS (
    SELECT DISTINCT Sub, Rank
    FROM df
    WHERE Event_name IN ('Obstructive Apnea', 'Hypopnea')
)

-- 2)  Join and aggregate
SELECT
    d.Sub,
    d.Pre_10,                              
    AVG(d.Sys_median) AS pre_mean_Systolic,
    AVG(d.O_median)   AS pre_mean_SpO2,
    AVG(d.HR_median)  AS pre_mean_HeartRate,
    AVG(d.Di_median)  AS pre_mean_Diastolic,

    MAX(d.Sys_median) AS pre_max_Systolic,
    MAX(d.O_median)   AS pre_max_SpO2,
    MAX(d.HR_median)  AS pre_max_HeartRate,
    MAX(d.Di_median)  AS pre_max_Diastolic,
    
    count(distinct Timestamp) as pre_event_duration    
    
FROM df            AS d
JOIN target_ranks  AS t
  ON d.Sub   = t.Sub
 AND d.Pre_10 = t.Rank                    
GROUP BY
    d.Sub,
    d.Pre_10
""").df().to_csv(
    '/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/tmp/pre_10.tsv',
    sep='\t',
    index=False            # no row numbers
)


In [3]:
#3. Post event data

duckdb.query("""
-- 1)  DISTINCT (Sub, Rank) for the two target event types
WITH target_ranks AS (
    SELECT DISTINCT Sub, Rank
    FROM df
    WHERE Event_name IN ('Obstructive Apnea', 'Hypopnea')
)

-- 2)  Join and aggregate
SELECT
    d.Sub,
    d.post_10,                             
    AVG(d.Sys_median) AS post_event_mean_Systolic,
    AVG(d.O_median)   AS post_event_SpO2,
    AVG(d.HR_median)  AS post_event_HeartRate,
    AVG(d.Di_median)  AS post_event_Diastolic,

    MAX(d.Sys_median) AS post_event_max_Systolic,
    MAX(d.O_median)   AS post_event_max_SpO2,
    MAX(d.HR_median)  AS post_event_max_HeartRate,
    MAX(d.Di_median)  AS post_event_max_Diastolic,    
    count(distinct Timestamp) as post_event_duration,    

    MIN(d.O_median)   AS post_event_min_SpO2
    
FROM df            AS d
JOIN target_ranks  AS t
  ON d.Sub   = t.Sub
 AND d.post_10 = t.Rank                    
GROUP BY
    d.Sub,
    d.post_10
""").df().to_csv(
    '/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/tmp/event_post_10_v1.tsv',
    sep='\t',
    index=False            # no row numbers
)


In [22]:
file_path = '/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/tmp/summary_clean_plot7'
df1 = pd.read_csv(file_path, delimiter='\t')



In [ ]:
# Clean heading and loading again

In [27]:
import duckdb

duckdb.query(r"""
WITH prep AS (
    SELECT
        Event_name,
        event_Sys_mode,
        event_Pos_median,

        /* ▸ duration bucket */
        CASE
            WHEN event_duration <  5 THEN '< 5 s'
            WHEN event_duration < 10 THEN '5–10 s'
            WHEN event_duration < 20 THEN '10–20 s'
            ELSE                        '≥ 20 s'
        END AS duration_cat,

        /* ▸ baseline SpO₂ bucket */
        CASE
            WHEN pre_mean_SpO2 > 100 THEN '> 100'
            WHEN pre_mean_SpO2 >  95 THEN '100–95'
            WHEN pre_mean_SpO2 >  90 THEN '95–90'
            ELSE                         '< 90'
        END AS pre_spo2_band,

        /* ▸ baseline SBP bucket (use pre_mean_Systolic!) */
        CASE
            WHEN pre_mean_Systolic <  90 THEN 'Hypotension (<90 mmHg)'
            WHEN pre_mean_Systolic BETWEEN 90 AND 139 THEN 'Normal (90–139 mmHg)'
            WHEN pre_mean_Systolic >= 140 THEN 'Hypertension (≥140 mmHg)'
            ELSE 'Unknown'
        END AS pre_sys_band,

        /* ▸ baseline HR bucket */
        CASE
            WHEN event_mean_HeartRate < 60 THEN 'Bradycardia (<60 bpm)'
            WHEN event_mean_HeartRate BETWEEN 60 AND 69 THEN 'Low Normal (60–69 bpm)'
            WHEN event_mean_HeartRate BETWEEN 70 AND 79 THEN 'Mid Normal (70–79 bpm)'
            WHEN event_mean_HeartRate BETWEEN 80 AND 100 THEN 'High Normal (80–100 bpm)'
            WHEN event_mean_HeartRate > 100 THEN 'Tachycardia (>100 bpm)'
            ELSE 'Unknown'
        END AS pre_hr_band,

        /* ▸ change scores (invalid ⇒ 0) */
        CASE
            WHEN post_event_max_Systolic IS NULL OR pre_mean_Systolic IS NULL
              OR post_event_max_Systolic = 0 OR pre_mean_Systolic = 0
            THEN 0
            ELSE post_event_max_Systolic - pre_mean_Systolic
        END AS delta_sys,

        CASE
            WHEN post_event_max_SpO2 IS NULL OR pre_mean_SpO2 IS NULL
              OR post_event_max_SpO2 = 0   OR pre_mean_SpO2 = 0
            THEN 0
            ELSE post_event_max_SpO2 - pre_mean_SpO2
        END AS delta_sp02,

        CASE
            WHEN post_event_min_SpO2 IS NULL OR pre_mean_SpO2 IS NULL
              OR post_event_min_SpO2 = 0   OR pre_mean_SpO2 = 0
            THEN 0
            ELSE post_event_min_SpO2 - pre_mean_SpO2
        END AS delta_sp02_drop,

        CASE
            WHEN post_event_max_HeartRate IS NULL OR event_mean_HeartRate IS NULL
              OR post_event_max_HeartRate = 0   OR event_mean_HeartRate = 0
            THEN 0
            ELSE post_event_max_HeartRate - event_mean_HeartRate
        END AS delta_HeartRate_spike,

        COALESCE(NULLIF(post_event_max_Systolic,0),0) AS post_event_max_Systolic,
        COALESCE(NULLIF(pre_mean_Systolic,0),0)       AS pre_mean_Systolic,
        COALESCE(NULLIF(pre_mean_SpO2,0),0)           AS pre_mean_SpO2,
        COALESCE(NULLIF(post_event_max_SpO2,0),0)     AS post_event_max_SpO2
    FROM df1
    WHERE Event_name IN ('Obstructive Apnea','Hypopnea')
),

banded AS (
    SELECT
        *,
        /* ▸ BP change band */
        CASE
            WHEN delta_sys >= 10 THEN '10+'
            WHEN delta_sys >  0 THEN '0_10'
            WHEN delta_sys =  0 THEN '0'
            ELSE                  '-5_0'
        END AS post_sys_range,

        /* ▸ SpO₂ change band */
        CASE
            WHEN delta_sp02 >= 10 THEN '10+'
            WHEN delta_sp02 >  0 THEN '0_10'
            WHEN delta_sp02 =  0 THEN '0'
            ELSE                   '-5_0'
        END AS post_sp02_range,

        /* ▸ SpO₂ drop band */
        CASE
            WHEN delta_sp02_drop >=   0 THEN '0+'
            WHEN delta_sp02_drop >= -10 THEN '-10'
            WHEN delta_sp02_drop >= -20 THEN '-20'
            ELSE NULL
        END AS post_delta_sp02_drop_unit,

        /* ▸ HR spike band (largest thresholds first) */
        CASE
            WHEN delta_HeartRate_spike >= 20 THEN '20+'
            WHEN delta_HeartRate_spike >= 10 THEN '10+'
            WHEN delta_HeartRate_spike >=  0 THEN '0+'
            ELSE NULL
        END AS post_delta_HeartRate_spike_range
    FROM prep
)

SELECT
    Event_name,
    event_Sys_mode,
    event_Pos_median,
    duration_cat,
    pre_spo2_band,
    pre_sys_band,
    pre_hr_band,
    post_sp02_range,
    post_sys_range,
    post_delta_HeartRate_spike_range,
    post_delta_sp02_drop_unit,
    COUNT(*) AS cnt
FROM banded
GROUP BY
    Event_name,
    event_Sys_mode,
    event_Pos_median,
    duration_cat,
    pre_spo2_band,
    pre_sys_band,
    pre_hr_band,
    post_sp02_range,
    post_sys_range,
    post_delta_HeartRate_spike_range,
    post_delta_sp02_drop_unit
""").df().to_csv(
    '/Users/ebenezer/Documents/SleepApnea/WorldSleepConf/tmp/temp14.tsv',
    sep='\t',
    index=False
)
